Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [173]:
subset = 100000
max_steps = 1001
regularization = 0.01
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:subset, :])
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.constant(train_labels[:subset, :])
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.add(tf.matmul(tf_train_dataset, weights), biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf_valid_dataset, weights), biases))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf_test_dataset, weights), biases))
    
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for logistic regression')
    
    for step in range(max_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized for logistic regression
Loss at step 0: 63.942646
Training accuracy: 11.5%
Validation accuracy: 12.8%
Loss at step 100: 10.632368
Training accuracy: 74.8%
Validation accuracy: 74.9%
Loss at step 200: 2.746022
Training accuracy: 80.0%
Validation accuracy: 79.9%
Loss at step 300: 1.154701
Training accuracy: 82.2%
Validation accuracy: 81.8%
Loss at step 400: 0.830739
Training accuracy: 82.6%
Validation accuracy: 82.2%
Loss at step 500: 0.763470
Training accuracy: 82.7%
Validation accuracy: 82.2%
Loss at step 600: 0.749275
Training accuracy: 82.7%
Validation accuracy: 82.2%
Loss at step 700: 0.746226
Training accuracy: 82.7%
Validation accuracy: 82.2%
Loss at step 800: 0.745548
Training accuracy: 82.7%
Validation accuracy: 82.3%
Loss at step 900: 0.745386
Training accuracy: 82.7%
Validation accuracy: 82.3%
Loss at step 1000: 0.745341
Training accuracy: 82.7%
Validation accuracy: 82.3%
Test accuracy: 89.0%


In [401]:
#NN section
batch_size = 128
max_steps = 3001
regularization = 0.0015
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    relu_size = 1024
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, relu_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
    biases_2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    logits = tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 848.164001
Training accuracy: 9.4%
Validation accuracy: 40.1%
Loss at step 500: 229.317474
Training accuracy: 75.8%
Validation accuracy: 80.4%
Loss at step 1000: 103.131088
Training accuracy: 85.2%
Validation accuracy: 82.9%
Loss at step 1500: 48.663277
Training accuracy: 83.6%
Validation accuracy: 84.7%
Loss at step 2000: 23.066505
Training accuracy: 87.5%
Validation accuracy: 87.0%
Loss at step 2500: 11.164434
Training accuracy: 89.8%
Validation accuracy: 87.2%
Loss at step 3000: 5.567663
Training accuracy: 89.1%
Validation accuracy: 87.8%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [242]:
#NN section
batch_size = 128
max_steps = 51
repeat_batch = 1
regularization = 0.0
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    relu_size = 1024
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, relu_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
    biases_2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    logits = tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step%repeat_batch * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 10 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 254.628922
Training accuracy: 10.2%
Validation accuracy: 30.3%
Loss at step 10: 0.000000
Training accuracy: 100.0%
Validation accuracy: 66.4%
Loss at step 20: 0.000000
Training accuracy: 100.0%
Validation accuracy: 66.4%
Loss at step 30: 0.000000
Training accuracy: 100.0%
Validation accuracy: 66.4%
Loss at step 40: 0.000000
Training accuracy: 100.0%
Validation accuracy: 66.4%
Loss at step 50: 0.000000
Training accuracy: 100.0%
Validation accuracy: 66.4%
Test accuracy: 72.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [293]:
#NN section
batch_size = 128
max_steps = 1001
regularization = 0.0
keep_prob = 0.3
repeat_batch = 2

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    relu_size = 1024
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, relu_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
    biases_2 = tf.Variable(tf.truncated_normal([num_labels]))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    logits = tf.add(tf.matmul(tf.nn.dropout(tf.nn.relu(layer_1),keep_prob), weights_2), biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2))
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step%repeat_batch * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 613.454956
Training accuracy: 11.7%
Validation accuracy: 32.9%
Loss at step 500: 0.000000
Training accuracy: 100.0%
Validation accuracy: 76.7%
Loss at step 1000: 0.000000
Training accuracy: 100.0%
Validation accuracy: 77.0%
Test accuracy: 84.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [426]:
#Single Layer
batch_size = 256
max_steps = 20001
regularization = 0.0015
relu_size = 1024
decay_steps = 1000
decay = 0.9
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, relu_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([relu_size, num_labels], stddev=np.sqrt(2/relu_size)))
    biases_2 = tf.Variable(tf.truncated_normal([num_labels], stddev=np.sqrt(2/relu_size)))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    logits = tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)))
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 467.753357
Training accuracy: 7.0%
Validation accuracy: 48.8%
Loss at step 1000: 104.927330
Training accuracy: 82.8%
Validation accuracy: 83.4%
Loss at step 2000: 25.235800
Training accuracy: 89.8%
Validation accuracy: 87.1%
Loss at step 3000: 6.711727
Training accuracy: 86.3%
Validation accuracy: 88.2%
Loss at step 4000: 2.237395
Training accuracy: 85.9%
Validation accuracy: 88.8%
Loss at step 5000: 0.942488
Training accuracy: 90.6%
Validation accuracy: 88.6%
Loss at step 6000: 0.582206
Training accuracy: 90.6%
Validation accuracy: 89.0%
Loss at step 7000: 0.470588
Training accuracy: 90.2%
Validation accuracy: 89.2%
Loss at step 8000: 0.510506
Training accuracy: 88.7%
Validation accuracy: 89.2%
Loss at step 9000: 0.521807
Training accuracy: 89.8%
Validation accuracy: 89.2%
Loss at step 10000: 0.539525
Training accuracy: 88.3%
Validation accuracy: 89.4%
Loss at step 11000: 0.430828
Training accuracy: 91.0%
Validation accuracy: 89.1%
Loss at st

In [446]:
#Two Layer
batch_size = 256
max_steps = 10001
regularization = 0.0015
l1_size = 1024
l2_size = 512
decay_steps = 1000
decay = 0.9
start_rate = 0.3
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, l1_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([l1_size, l2_size], stddev=np.sqrt(2/l1_size)))
    biases_2 = tf.Variable(tf.truncated_normal([l2_size]))
    
    weights_3 = tf.Variable(tf.truncated_normal([l2_size, num_labels], stddev=np.sqrt(2/l2_size)))
    biases_3 = tf.Variable(tf.truncated_normal([num_labels]))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    layer_2 = tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2)
    logits = tf.add(tf.matmul(tf.nn.relu(layer_2), weights_3), biases_3)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)) + tf.mul(regularization,tf.nn.l2_loss(weights_3)))
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(start_rate, global_step, decay_steps, decay)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2), weights_3), biases_3))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2), weights_3), biases_3))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 468.590607
Training accuracy: 9.0%
Validation accuracy: 10.0%
Loss at step 1000: 200.648834
Training accuracy: 9.0%
Validation accuracy: 10.0%
Loss at step 2000: 94.233925
Training accuracy: 11.3%
Validation accuracy: 10.0%
Loss at step 3000: 48.319599
Training accuracy: 7.4%
Validation accuracy: 10.0%
Loss at step 4000: 26.986076
Training accuracy: 8.6%
Validation accuracy: 10.0%
Loss at step 5000: 16.395134
Training accuracy: 10.2%
Validation accuracy: 10.0%
Loss at step 6000: 10.811361
Training accuracy: 9.8%
Validation accuracy: 10.0%
Loss at step 7000: 5.972039
Training accuracy: 85.9%
Validation accuracy: 81.3%
Loss at step 8000: 4.110205
Training accuracy: 85.2%
Validation accuracy: 84.4%
Loss at step 9000: 2.958159
Training accuracy: 88.3%
Validation accuracy: 85.1%
Loss at step 10000: 2.314628
Training accuracy: 87.9%
Validation accuracy: 86.5%
Test accuracy: 92.7%


In [455]:
#Three Layer
batch_size = 128
max_steps = 10001
regularization = 0.0025
l1_size = 1024
l2_size = 512
l3_size = 256
decay_steps = 500
decay = 0.9
start_rate = 0.5
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, l1_size]))
    biases_1 = tf.Variable(tf.zeros([relu_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([l1_size, l2_size], stddev=np.sqrt(2/l1_size)))
    biases_2 = tf.Variable(tf.truncated_normal([l2_size]))
    
    weights_3 = tf.Variable(tf.truncated_normal([l2_size, l3_size], stddev=np.sqrt(2/l2_size)))
    biases_3 = tf.Variable(tf.truncated_normal([l3_size]))
    
    weights_4 = tf.Variable(tf.truncated_normal([l3_size, num_labels], stddev=np.sqrt(2/l3_size)))
    biases_4 = tf.Variable(tf.truncated_normal([num_labels]))
    
    layer_1 = tf.add(tf.matmul(tf_train_dataset, weights_1), biases_1)
    layer_2 = tf.add(tf.matmul(tf.nn.relu(layer_1), weights_2), biases_2)
    layer_3 = tf.add(tf.matmul(tf.nn.relu(layer_2), weights_3), biases_3)
    logits = tf.add(tf.matmul(tf.nn.relu(layer_3), weights_4), biases_4)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels) + tf.mul(regularization,tf.nn.l2_loss(weights_1)) + tf.mul(regularization,tf.nn.l2_loss(weights_2)) + tf.mul(regularization,tf.nn.l2_loss(weights_3)) + tf.mul(regularization,tf.nn.l2_loss(weights_4)))
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(start_rate, global_step, decay_steps, decay)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    
    train_prediction = tf.nn.softmax(logits)
    
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.add(tf.matmul(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights_1), biases_1)), weights_2), biases_2), weights_3), biases_3), weights_4), biases_4))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.add(tf.matmul(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights_1), biases_1)), weights_2), biases_2), weights_3), biases_3), weights_4), biases_4))
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('Initialized for 1 layer NN')
    
    for step in range(max_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized for 1 layer NN
Loss at step 0: 779.936768
Training accuracy: 10.2%
Validation accuracy: 10.0%
Loss at step 1000: 83.782898
Training accuracy: 10.9%
Validation accuracy: 10.0%
Loss at step 2000: 14.824141
Training accuracy: 36.7%
Validation accuracy: 23.7%
Loss at step 3000: 3.492973
Training accuracy: 86.7%
Validation accuracy: 84.7%
Loss at step 4000: 1.497466
Training accuracy: 85.2%
Validation accuracy: 85.8%
Loss at step 5000: 0.938557
Training accuracy: 85.2%
Validation accuracy: 86.7%
Loss at step 6000: 0.606275
Training accuracy: 89.1%
Validation accuracy: 87.3%
Loss at step 7000: 0.689258
Training accuracy: 83.6%
Validation accuracy: 86.6%
Loss at step 8000: 0.703315
Training accuracy: 85.2%
Validation accuracy: 88.1%
Loss at step 9000: 0.758294
Training accuracy: 79.7%
Validation accuracy: 88.0%
Loss at step 10000: 0.646708
Training accuracy: 85.9%
Validation accuracy: 88.7%
Test accuracy: 94.1%
